In [ ]:
class SentenceTransformer(Ranker):

    def __init__(self, model_name: str = "all-MiniLM-L6-v2", device: torch.device = None, use_amp: bool = False):
        self.device = get_device() if device is None else device
        self.use_amp = use_amp

        self.model = SentenceTransformer(model_name, device=self.device, trust_remote_code=True)

    def run(self, dataset: MSMarcoDataset, query_id: str, score_docs: list[tuple[str, float]], k: int = 10, **kwargs) -> list[tuple[str, float]]:
        query = dataset.queries[query_id]
        documents = [dataset.documents[doc_id] for doc_id, _ in score_docs]

        # Encode query and documents
        embeddings = self.model.encode(
            [query] + documents,
            normalize_embeddings=self.normalize_embeddings,
            convert_to_tensor=True,
            show_progress_bar=False  # Disable tqdm progress bar
        )
        query_emb = embeddings[0]
        document_embs = embeddings[1:]

        # Compute cosine similarity scores
        scores = torch.nn.functional.cosine_similarity(
            query_emb.unsqueeze(0), document_embs
        ).tolist()

        new_score_docs = [(doc_id, score) for (doc_id, _), score in zip(score_docs, scores)]
        new_score_docs = sorted(new_score_docs, key=lambda x: x[1], reverse=True)[:k]

        return new_score_docs